In [134]:
import tensorflow as tf
import torch
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, ElectraTokenizerFast, ElectraForMaskedLM, \
    ElectraForPreTraining
import numpy as np

In [80]:
VOCAB_FILE = '../../data/bitcoin_twitter-vocab.txt'
tokenizer_custom = {
    '@HTAG': '[HTAG]',
    '@USR': '[USR]',
    '@CURR': '[CURR]',
    '@EMOJI': '[EMOJI]',
    '@URL': '[URL]',
    '@TIME': '[TIME]',
    '@DATE': '[DATE]',
    '@NUM': '[NUM]',
    '@MASK': '[MASK]'
}
additional_tokens = list(tokenizer_custom.values())

tokenizer = ElectraTokenizerFast(vocab_file=VOCAB_FILE)
tokenizer.add_special_tokens({
    'additional_special_tokens': additional_tokens
})

0

In [139]:
MODEL_DIR = '../../data/models/discriminator'
model = ElectraForMaskedLM.from_pretrained(MODEL_DIR)

Some weights of the model checkpoint at ../../data/models/discriminator were not used when initializing ElectraForMaskedLM: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at ../../data/models/discriminator and are newly initialized: ['generator_predictions.LayerNorm.weight', 'generator_predictions.LayerNor

In [141]:
sequence = f'Just bought [CURR] {tokenizer.mask_token}. Can not wait until it gets to [NUM] {tokenizer.mask_token}'
sequence = f'well [HTAG] {tokenizer.mask_token} is taking over . go get [CURR] ethereum [CURR] chainlink [CURR] bitcoin and your good .'
sequence = f'Are you a real {tokenizer.mask_token} .'

input_ids = tokenizer(sequence, return_tensors='pt')['input_ids']
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
print(mask_token_index)

token_logits = model(input_ids)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

tensor([5])
Are you a real parabol .
Are you a real psychological .
Are you a real ##late .
Are you a real ##nal .
Are you a real bron .


In [138]:
type(model.get_input_embeddings())

torch.nn.modules.sparse.Embedding

In [150]:
emb = model.get_input_embeddings()
embeddings=np.zeros((16537,128))
words = []
i = 0
for word,index in tokenizer.get_vocab().items():
    words.append(word)
    embeddings[i]=emb(torch.tensor(index)).detach().numpy()
    i += 1

In [151]:
np.savetxt("foo.tsv", embeddings, delimiter="\t")

In [155]:
with open('labels.tsv', 'w') as f:
    for w in words:
        f.write(f'{w}\n')